#Dependencies

In [1]:
! pip install -q tensorflow-text
! pip install tensorflow_addons
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.9 MB/s eta 0:00:00


In [2]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Imports

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import tqdm
pd.set_option('display.width', 3000)
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import keras
from keras import regularizers
import re
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm_notebook
tqdm_notebook().pandas()
import tqdm
#from pandarallel import pandarallel
from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0it [00:00, ?it/s]

#Prepare codeswitch data

In [6]:
import pandas as pd
df_dic = pd.read_csv("./drive/MyDrive/Thesis/Dictionary.csv")

In [7]:
df_dic.head(5)

,Arabic,English
0,سبب,reason
1,تكرار,repeating
2,التجربة,the experiment
3,ستحتاج,needs
4,النتائج,results


In [8]:
arabic = df_dic.Arabic.values
english = df_dic.English.values
english.shape

(30,)

In [9]:
def switch(text):
  try:
    new = []   
    text = text.split() 
    for word in text :
     translation = ""
     for i in range(0,len(arabic)) :
       if word == arabic[i] :
         translation = english[i]
         break       
     if translation != "" :
       new.append(translation)
     else:
       new.append(word)
  except: 
          pass

  newText = ' '.join([str(elem) for i,elem in enumerate(new)])
  return newText



In [10]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [11]:
def CountEnglish(text):
  english_words = 0
  try:
    text = text.split()  
    for word in text :
      if(isEnglish(word)) :
        english_words += 1
  except: 
          pass

  return english_words

def CountTotalW(text):
  words = 0
  try:
    text = text.split()  
    for word in text :
      words += 1
  except: 
          pass
          
  return words


In [12]:
def sum(numm) :
  r = 0
  for i in range(len(numm)) :
    r+=numm[i]
  return r

In [13]:
import string,re
punctuation = string.punctuation + "؟'«»٫٬،.؛"
def remove_punctuation(text):
  return text.translate(str.maketrans('', '', punctuation))
def remove_extra_space(text):
  return re.sub(' +', ' ', text)

#Read data

Run on Whole Data

In [20]:

def whole_data():
  df = pd.read_csv("./drive/MyDrive/Thesis/sas-ar.csv")
  df.dropna(inplace=True)


  df['EssayText'] = df['EssayText'].progress_apply(lambda text: switch(text))

  newAns = df.EssayText.values
  df['English_Words'] = df['EssayText'].progress_apply(lambda text: CountEnglish(text))
  df['Total_Words'] = df['EssayText'].progress_apply(lambda text: CountTotalW(text))
  
  CodeSwitch_Percentage = []
  englishWords = df.English_Words.values
  totalWords = df.Total_Words.values
  CodeSwitch_Percentage.append( (englishWords/totalWords)*100  )
  codeSwitchPercentages = CodeSwitch_Percentage[0]
  df['CodeSwitch_Percentage'] = codeSwitchPercentages


  English = sum(englishWords)
  Words = sum(totalWords)
  print("English is : " , English," out of total number of words: ", Words)
  print("CodeSwitch percentage is : ",  (English/Words)*100 , "%" )

  
  df["EssayText"] = df["EssayText"].apply(lambda text: remove_punctuation(text))
  df["EssayText"] = df["EssayText"].apply(lambda text: remove_extra_space(text))


  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
  test = df[df['EssaySet']==1]
  X = np.array(embed(test["EssayText"]))
  for i in range(2,11):
    test = df[df['EssaySet']==i]
    curr = np.array(embed(test["EssayText"]))
    X = np.concatenate((X,curr), axis=0)
  y = np.array(df["Score1"])
  essaySet = np.array(df['EssaySet'])
  essaySet = np.expand_dims(essaySet,axis=1)/100
  X = np.concatenate((essaySet,X), axis=1)  
  X = np.expand_dims(X,axis=1)

 
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df[["EssaySet","Score1"]] ,test_size=0.2, random_state=42)
  print(X_train.shape)
  return X_train, X_test, y_train, y_test


In [21]:
X_train, X_test, y_train, y_test = whole_data()

  0%|          | 0/17043 [00:00<?, ?it/s]

  0%|          | 0/17043 [00:00<?, ?it/s]

  0%|          | 0/17043 [00:00<?, ?it/s]

English is :  51623  out of total number of words:  617962
CodeSwitch percentage is :  8.353749906952206 %
(13634, 1, 513)


In [22]:
X_train.shape

(13634, 1, 513)

#Run Individually

In [23]:
def read_csv(question):
  df = pd.read_csv("./drive/MyDrive/Thesis/sas-ar.csv")
  df.dropna(inplace=True)
  df = df[df['EssaySet']==question]
  return df
  

In [24]:
def embed_text(df):
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
  # test = df[df['EssaySet']==1]
  X = np.array(embed(df["EssayText"]))
  print(X.shape)  
  y = np.array(df["Score1"])
  return X,y

#Preprocess Data

In [25]:
df = read_csv(10)

df["EssayText"] = df["EssayText"].apply(lambda text: remove_punctuation(text))
df["EssayText"] = df["EssayText"].apply(lambda text: remove_extra_space(text))

df['EssayText'] = df['EssayText'].progress_apply(lambda text: switch(text))
newAns = df.EssayText.values
df['English_Words'] = df['EssayText'].progress_apply(lambda text: CountEnglish(text))
df['Total_Words'] = df['EssayText'].progress_apply(lambda text: CountTotalW(text))

CodeSwitch_Percentage = []
englishWords = df.English_Words.values
totalWords = df.Total_Words.values
CodeSwitch_Percentage.append( (englishWords/totalWords)*100  )
codeSwitchPercentages = CodeSwitch_Percentage[0]
df['CodeSwitch_Percentage'] = codeSwitchPercentages


Words = sum(totalWords)
English = sum(englishWords)

print("English is : " , English," out of total number of words: ", Words)
print("CodeSwitch percentage is : ",  (English/Words)*100 , "%" )



  0%|          | 0/1640 [00:00<?, ?it/s]

  0%|          | 0/1640 [00:00<?, ?it/s]

  0%|          | 0/1640 [00:00<?, ?it/s]

English is :  10717  out of total number of words:  57037
CodeSwitch percentage is :  18.78955765555692 %


In [26]:
X,y = embed_text(df)
X = np.expand_dims(X,axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y ,test_size=0.2, random_state=42)
print(np.unique(y_train, return_counts=True))

(1640, 512)
(array([0, 1, 2]), array([232, 616, 464]))


In [27]:
df.shape

(1640, 8)

In [28]:
newAns = df.EssayText.values

#Convert Embeddings from 3D to 2D


In [29]:
nsamples, nx, ny = X_train.shape
d2_train_dataset = X_train.reshape((nsamples,nx*ny))
d2_train_dataset.shape

(1312, 512)

In [30]:
nsamples_t, nx_T, ny_t = X_test.shape
d2_test_dataset = X_test.reshape((nsamples_t,nx_T*ny_t))
d2_test_dataset.shape

(328, 512)

#Descison Tree

In [37]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [38]:
X_train.shape

(1312, 1, 512)

In [39]:
nsamples, nx, ny = X_train.shape
d2_train_dataset = X_train.reshape((nsamples,nx*ny))
d2_train_dataset.shape

(1312, 512)

In [40]:
nsamples_t, nx_T, ny_t = X_test.shape
d2_test_dataset = X_test.reshape((nsamples_t,nx_T*ny_t))
d2_test_dataset.shape

(328, 512)

In [41]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(d2_train_dataset,y_train)

#Predict the response for test dataset
y_pred = clf.predict(d2_test_dataset)


In [42]:
cohen_kappa_score(y_test, y_pred, weights='quadratic')

0.4691512571980401

#SVC

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf = clf.fit(d2_train_dataset,y_train)
y_pred = clf.predict(d2_test_dataset)

In [36]:
cohen_kappa_score(y_test, y_pred, weights='quadratic')

0.7021478205938092

#KNeighborsClassifier

In [43]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh = neigh.fit(d2_train_dataset,y_train)
y_pred = neigh.predict(d2_test_dataset)
cohen_kappa_score(y_test, y_pred, weights='quadratic')

0.689957652752571

#*RandomForestClassifier*

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf = clf.fit(d2_train_dataset,y_train)
y_pred = clf.predict(d2_test_dataset)

cohen_kappa_score(y_test, y_pred, weights='quadratic')

0.475063272294179

#Lazy CLassifier

In [31]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from lazypredict.Supervised import LazyClassifier

In [33]:
clf=LazyClassifier(verbose=0,ignore_warnings=True,custom_metric=None)
models,predictions=clf.fit(d2_train_dataset,d2_test_dataset,y_train,y_test)


100%|██████████| 29/29 [01:16<00:00,  2.64s/it]


In [34]:
print(models)

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  Time Taken
Model                                                                                   
SVC                                0.77               0.72    None      0.76        0.32
XGBClassifier                      0.76               0.71    None      0.75       10.39
LGBMClassifier                     0.75               0.70    None      0.75       28.11
NuSVC                              0.76               0.70    None      0.75        0.40
ExtraTreesClassifier               0.75               0.68    None      0.74        0.99
NearestCentroid                    0.68               0.68    None      0.68        0.05
LogisticRegression                 0.70               0.68    None      0.70        0.32
SGDClassifier                      0.70               0.67    None      0.70        0.34
GaussianNB                         0.68               0.67    None      0.68        0.06
RandomForestClassifie